<a href="https://colab.research.google.com/github/SoroushGhaderi/Dry_Bean_Machine_Learning/blob/master/Dry_Bean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [47]:
DATA_PATH = "/content/drive/MyDrive/Dry_Bean/data/Dry_Bean_Dataset.xlsx"
dry_beans = pd.read_excel(DATA_PATH)
dry_beans.head()

,Area,Perimeter,MajorAxisLength,MinorAxisLength,AspectRation,Eccentricity,ConvexArea,EquivDiameter,Extent,Solidity,roundness,Compactness,ShapeFactor1,ShapeFactor2,ShapeFactor3,ShapeFactor4,Class
0,28395,610.291,208.178117,173.888747,1.197191,0.549812,28715,190.141097,0.763923,0.988856,0.958027,0.913358,0.007332,0.003147,0.834222,0.998724,SEKER
1,28734,638.018,200.524796,182.734419,1.097356,0.411785,29172,191.272750,0.783968,0.984986,0.887034,0.953861,0.006979,0.003564,0.909851,0.998430,SEKER
2,29380,624.110,212.826130,175.931143,1.209713,0.562727,29690,193.410904,0.778113,0.989559,0.947849,0.908774,0.007244,0.003048,0.825871,0.999066,SEKER
3,30008,645.884,210.557999,182.516516,1.153638,0.498616,30724,195.467062,0.782681,0.976696,0.903936,0.928329,0.007017,0.003215,0.861794,0.994199,SEKER
4,30140,620.134,201.847882,190.279279,1.060798,0.333680,30417,195.896503,0.773098,0.990893,0.984877,0.970516,0.006697,0.003665,0.941900,0.999166,SEKER


In [72]:
class DryBeanDataset(Dataset):
    def __init__(self, path):
        data = pd.read_excel(path)
        self.X = torch.from_numpy(np.array(data.iloc[:, :-1], dtype=np.float32))
        self.label_encoder = LabelEncoder()
        self.y = torch.from_numpy(self.label_encoder.fit_transform(data.iloc[:, -1]))
        self.y_label_classes = self.label_encoder.classes_
        self.n_samples = data.shape[0]
    
    def __getitem__(self, item):
        return self.X[item], self.y[item]

    def __len__(self):
        return self.n_samples
    
    def inverse_encoder(self):
        return self.y_label_classes

In [73]:
dry_bean = DryBeanDataset(DATA_PATH)

In [79]:
features, labels = dry_bean[:]

In [75]:
dry_bean.inverse_encoder( )

array(['BARBUNYA', 'BOMBAY', 'CALI', 'DERMASON', 'HOROZ', 'SEKER', 'SIRA'],
      dtype=object)

In [89]:
pd.Series(labels).value_counts()

3    3546
6    2636
5    2027
4    1928
2    1630
0    1322
1     522
dtype: int64